In [2]:
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map, load_checkpoint_in_model
from transformers import AutoConfig, AutoModelForCausalLM
from huggingface_hub import snapshot_download
import torch


/home/fsuser/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# checkpoint = "facebook/opt-30b"
# checkpoint = 'intlsy/opt-175b-hyperparam'
# checkpoint = "facebook/opt-125m"
checkpoint = "facebook/opt-6.7b"
weights_location = snapshot_download(checkpoint, allow_patterns=["*.bin", 'pytorch_model.bin.index.json'])#hf_hub_download(checkpoint, "*.bin")

with init_empty_weights():
    config = AutoConfig.from_pretrained(checkpoint)
    model = AutoModelForCausalLM.from_config(config)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
device_map = infer_auto_device_map(model)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [ ]:
device_map

In [4]:
# device_map['model.decoder.embed_tokens'] = 'disk'

In [5]:
device_map

{'model.decoder.embed_tokens': 'cpu',
 'lm_head': 'cpu',
 'model.decoder.embed_positions': 'cpu',
 'model.decoder.final_layer_norm': 'cpu',
 'model.decoder.layers.0': 'cpu',
 'model.decoder.layers.1': 'cpu',
 'model.decoder.layers.2': 'cpu',
 'model.decoder.layers.3': 'cpu',
 'model.decoder.layers.4': 'cpu',
 'model.decoder.layers.5': 'cpu',
 'model.decoder.layers.6': 'cpu',
 'model.decoder.layers.7': 'cpu',
 'model.decoder.layers.8': 'cpu',
 'model.decoder.layers.9': 'cpu',
 'model.decoder.layers.10': 'cpu',
 'model.decoder.layers.11': 'cpu',
 'model.decoder.layers.12': 'cpu',
 'model.decoder.layers.13': 'cpu',
 'model.decoder.layers.14': 'cpu',
 'model.decoder.layers.15.self_attn': 'cpu',
 'model.decoder.layers.15.activation_fn': 'cpu',
 'model.decoder.layers.15.self_attn_layer_norm': 'cpu',
 'model.decoder.layers.15.fc1': 'disk',
 'model.decoder.layers.15.fc2': 'disk',
 'model.decoder.layers.15.final_layer_norm': 'disk',
 'model.decoder.layers.16': 'disk',
 'model.decoder.layers.17'

In [7]:
m = AutoModelForCausalLM.from_pretrained(checkpoint, device_map=device_map, offload_folder='./offload_folder')
# model = load_checkpoint_and_dispatch( 
#     model, device_map=device_map,
#     offload_folder='./offload_folder',
#     checkpoint=weights_location,
#     max_memory={'cpu':'2GiB'}
# )

# model = load_checkpoint_in_model( 
#     model, device_map=device_map,
#     offload_folder='./offload_folder',
#     checkpoint=weights_location,
#     # max_memory={'cpu':'2GiB'}
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

: 

In [ ]:
m.hf_device_map

{'': 'cpu'}